In [40]:
### Library Imports
import pandas as pd
import numpy as np
import math 
import seaborn as sns 
import matplotlib.pyplot as plt
from datetime import datetime
import yfinance as yf 
import matplotlib.dates as mdates


In [41]:
### Load in Stock Data
tickers = ['GOOG', 'AAPL']
start_date = '2018-01-01'

def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    # Reset index to make headings in the same row
    data.reset_index(inplace = True)

    data['Date'] = pd.to_datetime(data['Date'])

    return data

stock_data = import_stock_data(tickers, start_date)
stock_data


[*********************100%%**********************]  2 of 2 completed


,Date,GOOG,AAPL
0,2018-01-02,40.670982,53.250000
1,2018-01-03,40.663887,54.124001
2,2018-01-04,40.852768,54.320000
3,2018-01-05,41.317894,55.111500
4,2018-01-08,41.164433,55.347000
...,...,...,...
1576,2024-04-09,169.669998,158.139999
1577,2024-04-10,167.779999,157.660004
1578,2024-04-11,175.039993,160.789993
1579,2024-04-12,176.550003,159.190002


In [43]:
### MACD Function for Multiple Stocks
def get_macd(tickers, df):
    # Initialize empty DataFrame to store MACD data
    macd_df = pd.DataFrame(index=df.index)
    macd_vals = []

    # Loop for MACD calculation and df population
    for t in tickers:
        # Calculate 12-day Exponential Moving Averages (EMA)
        ema_12 = df[t].ewm(span=12, adjust=False).mean()
        # Calculate 26-day Exponential Moving Averages (EMA)
        ema_26 = df[t].ewm(span=26, adjust=False).mean()
        # Calculate MACD line (12-day EMA minus 26-day EMA)
        macd = ema_12 - ema_26
        # Add MACD column to the DataFrame
        macd_df[f'MACD_{t}'] = macd
        # Append last row values to macd_vals list
        macd_vals.append(macd.iloc[-1])

    return macd_df, macd_vals

macd_df, macd_vals = get_macd(tickers, stock_data)
print(macd_df)
print("MACD values list: ", macd_vals)

      MACD_GOOG  MACD_AAPL
0      0.000000   0.000000
1     -0.000566   0.069721
2      0.014064   0.139186
3      0.062471   0.255164
4      0.087442   0.361908
...         ...        ...
1576  -2.390475   3.643262
1577  -2.418451   3.708026
1578  -1.833665   3.966196
1579  -1.234146   3.995633
1580  -1.058293   3.745014

[1581 rows x 2 columns]
MACD values list:  [-1.0582931758821985, 3.7450139093668895]
